## Configuração inicial

In [1]:
import pickle
import pandas as pd

from Utils import templates, utils, recommender

In [ ]:
## Configurações base

wandb_key = "b9d02b504b9d57ae7801f351e79da0162387f010"

config = {
    #"runtime": "ROCm llama.cpp v1.23.0", 
    #"runtime": "CPU llama.cpp v1.22.2", # performance ruim
    "LLM_runtime": "Vulkan llama.cpp v1.23.0", # melhor opção
    "dataset": "ml_100k",
    "nsu" : 18,     # número de usuários para filtragem colaborativa    | Default :12
    "nci" :24,      # número de itens para filtragem colaborativa       | Default :19
    "lenlimit" : 24,  # limite de tamanho para a lista filmes assistidos  | Default : 8 | fazer o teste com o numero maximo de filmes 
    "test_run" : 20, # define a quantidade de recomendações, '0' para todos
    "obs": "nada"
}

## define o prompt template
prompt_template = templates.PROMPT_TEMPLATE_2
config.update({"prompt_template": prompt_template})

## define o prompt para formatar a resposta final 
#prompt_format = templates.PROMPT_TEMPLATE_ESTRUCTURE
prompt_format = "" # para não utilizar 
config.update({"prompt_format": prompt_format})

# load movie lens 100k dataset
dataset = utils.read_json("Data/ML100K_clean.json")
print(f'Quantidade de Usuários: {len(dataset)}')

Quantidade de Usuários: 943


---------   
## Escolha do modelo

### gemma-3-4b-it

In [3]:
config.update({
    "model_name" :"gemma-3-4b-it",
    "Arch" : "gemma3",
    "Quantization" : "Q4_K_M",
    "Temperature": 0.1,
    "max_tokens" : -1,  # Default : 4096
    "GPU Offload": 34,
    "CPU Thread Pool Size": 6,
    "Evaluation Batch Size": 512,
    "Flash Attention": False, # não vi vantagem no uso 
})

### meta-llama-3.1-8b-instruct

In [ ]:
config.update({
    "model_name" :"meta-llama-3.1-8b-instruct",
    "Arch" : "llama",
    "Quantization" : "Q4_K_M",
    "Temperature": 0.1,
    "max_tokens" : 4096,
    "GPU Offload": 34,
    "CPU Thread Pool Size": 6,
    "Evaluation Batch Size": 512,
    "Flash Attention": False,
})

### llama-3.2-3b-instruct

In [ ]:
config.update({
    "model_name" :"llama-3.2-3b-instruct",
    "Arch" : "llama",
    "Quantization" : "Q8_0",
    "Temperature": 0,
    "max_tokens" : -1, #Default :4096
    "GPU Offload": 34,
    "CPU Thread Pool Size": 6,
    "Evaluation Batch Size": 512,
    "Flash Attention": False,
})

In [ ]:
# retorna apenas o usuários que tem o gt no candidatos.
# implementar sobre o pipeline do projeto 

def get_candidate_ids_list(data, id_list, user_matrix_sim, num_u, num_i):
    cand_ids = []
    for i in id_list:
        watched_movies = data[i][0].split(' | ')
        candidate_items = utils.sort_user_filtering_items(data, watched_movies, user_matrix_sim[i], num_u, num_i)
        if data[i][-1] in candidate_items:
            cand_ids.append(i)
    return cand_ids

In [ ]:
id_list = list(range(0, len(dataset)))
#assert(len(id_list) == 943) # aqui é verificado se a lista possue exatamente essa quantidade

# Building indexes and similarity matrices for users and movies.
movie_idx = utils.build_moviename_index_dict(dataset)
user_sim_matrix = utils.build_user_similarity_matrix(dataset, movie_idx)


# para fazer a filtragem sobre os filmes 
#pop_dict = utils.build_movie_popularity_dict(dataset) 
#item_sim_matrix = utils.build_item_similarity_matrix(dataset)

## Execução

In [4]:
result_pkl = recommender.recommendation_workflow(config         = config,
                                                 wandb_key      = wandb_key,
                                                 dataset        = dataset,
                                                 prompt_template= prompt_template,
                                                 prompt_format  = prompt_format,
                                                 run_wandb      = False)

Processando:   0%|          | 0/20 [00:00<?, ?it/s]



teste : 8
GT: Casablanca
Predictions: 1. Braveheart
2. Schindler’s List
3. Chinatown
4. Amadeus
5. Casablanca
6. Vertigo
7. Fargo
8. Pulp Fiction
9. Taxi Driver
10. Citizen Kane
Query: 1. Braveheart
2. Schindler’s List
3. Chinatown
4. Amadeus
5. Casablanca
6. Vertigo
7. Fargo
8. Pulp Fiction
9. Taxi Driver
10. Citizen Kane
Relevants: Casablanca
recommendations_set: {'AMADEUS', 'CITIZEN KANE', 'TAXI DRIVER', 'SCHINDLER’S LIST', 'FARGO', 'CHINATOWN', 'BRAVEHEART', 'PULP FICTION', 'VERTIGO', 'CASABLANCA'}
ground_truth_set: {'CASABLANCA'}
recommendations_set: {'AMADEUS', 'CITIZEN KANE', 'TAXI DRIVER', 'SCHINDLER’S LIST', 'FARGO', 'CHINATOWN', 'BRAVEHEART', 'PULP FICTION', 'VERTIGO', 'CASABLANCA'}
ground_truth_set: {'CASABLANCA'}
Metrics for @5 and @10: {'hit@5': 0, 'precision@5': 0.0, 'recall@5': 0.0, 'ndcg@5': 0.0, 'hit@10': 1, 'precision@10': 0.1, 'recall@10': 1.0, 'ndcg@10': 0.2890648263178879}


teste : 11
GT: Sleepless in Seattle
Predictions: 1.  The Usual Suspects
2.  Die Hard
3.  

## Resultados

In [ ]:
with open(f'{result_pkl}', 'rb') as f:
    data = pickle.load(f)

In [ ]:
results = []
for key, value in data.items():
    if isinstance(key, int) and isinstance(value, dict):  # Pegando apenas os experimentos
        results.append({
            'Candidates': value.get('candidate_set', ''),
            'Ground Truth': value.get('ground_truth', ''),
            'gt_in_candidate_set': value.get('gt_in_candidate_set', ''),
            #'Input 1': value.get('input_1', ''),
            #'Predictions 1': value.get('predictions_1', ''),
            #'Input 2': value.get('input_2', ''),
            #'Predictions 2': value.get('predictions_2', ''),
            'Input 3': value.get('input_3', ''),
            'Predictions 3': value.get('predictions_3', ''),
            #'Recommendations': value.get('recommendations', ''),
            'rec_HitRate@10': value.get('rec_HitRate@10', ''),
            #'Precision': value.get('precision', ''),
            #'Recall': value.get('recall', ''),
            'rec_NDCG@10': value.get('rec_NDCG@10', '')
        })

df_results = pd.DataFrame(results)

df_results